In [36]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import json
from selenium.webdriver.common.keys import Keys
import time

In [60]:
species_list = ['African_Violet', 'Aloe_Vera', 'Anthurium', 
                'Dypsis lutescens', 'Asparagus Fern', 'Begonia', 
                "Bird's Nest", 'Strelitzia reginae', 'Nephrolepis_exaltata', 
                'Calathea', 'Aspidistra_elatior', "Aglaonema 'Cutlass'a", 'Chinese_Money_Plant_Pilea_peperomioides',
                  'Schlumbergera buckleyi', "Chrysanthemum 'Fireworks Igloo", 'Ctenanthe', "Narcissus 'Ambergate'", 'Dracaena marginata', 
                  'Dumb Cane', 'Alocasia', 'Hedera Helix', 'Hyacinth', 
                  'Iron_Cross_begonia', 'Crassula_ovata', 'Kalanchoe', 'Hemerocallis', 
                  'Convallaria', 'Pachira_aquatica', 'Monstera_Deliciosa', 
                  'Moth Orchid', 'Parlor_Palm', 'Peace_lily', 'Poinsettia', 'Polka_Dot', 
                  'Beaucarnea', 'Pothos', 'Prayer_Plant', 'Rattlesnake', 
                  'Rubber_Plant', 'Sago Palm', 'Umbrella Plant', 'Sanseviera pattens', 'Inch Plant', 'kaufmanniana tulip', 
                  'Venus Fly Trap', 'Spineless Yucca', 'ZZ Plant'
]

In [61]:
mapping = ['African_Violet_Saintpaulia_ionantha', 'Aloe_Vera', 'Anthurium_Anthurium_andraeanum', 
                'Areca_Palm_Dypsis_lutescens', 'Asparagus_Fern_Asparagus_setaceus', 'Begonia_Begonia_spp', 
                'Birds_Nest_Fern_Asplenium_nidus', 'Bird_of_Paradise_Strelitzia_reginae', 'Boston_Fern_Nephrolepis_exaltata', 
                'Calathea', 'Cast_Iron_Plant_Aspidistra_elatior', 'Chinese_evergreen_Aglaonema', 'Chinese_Money_Plant_Pilea_peperomioides',
                  'Christmas_Cactus_Schlumbergera_bridgesii', 'Chrysanthemum', 'Ctenanthe', 'Daffodils_Narcissus_spp', 'Dracaena', 
                  'Dumb_Cane_Dieffenbachia_spp', 'Elephant_Ear_Alocasia_spp', 'English_Ivy_Hedera_helix', 'Hyacinth_Hyacinthus_orientalis', 
                  'Iron_Cross_begonia_Begonia_masoniana', 'Jade_plant_Crassula_ovata', 'Kalanchoe', 'Lilium_Hemerocallis', 
                  'Lily_of_the_valley_Convallaria_majalis', 'Money_Tree_Pachira_aquatica', 'Monstera_Deliciosa_Monstera_deliciosa', 
                  'Orchid', 'Parlor_Palm_Chamaedorea_elegans', 'Peace_lily', 'Poinsettia_Euphorbia_pulcherrima', 'Polka_Dot_Plant_Hypoestes_phyllostachya', 
                  'Ponytail_Palm_Beaucarnea_recurvata', 'Pothos_Ivy_arum', 'Prayer_Plant_Maranta_leuconeura', 'Rattlesnake_Plant_Calathea_lancifolia', 
                  'Rubber_Plant_Ficus_elastica', 'Sago_Palm_Cycas_revoluta', 'Schefflera', 'Snake_plant_Sanseviera', 'Tradescantia', 'Tulip', 
                  'Venus_Flytrap', 'Yucca', 'ZZ_Plant_Zamioculcas_zamiifolia'
]

Selenium part:

In [ ]:

# Dictionary to store species data
species_data = {}

# Loop through each species in the list
for species_name in species_list:
    try:
        # Navigate to the plant species database page
        url = 'https://perenual.com/plant-species-database-search-finder'
        driver.get(url)

        # Wait for the search input element
        search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'input[placeholder="Search For A Plant"]'))
        )

        # Enter the species name into the search input
        search_input.send_keys(species_name.replace('_', ' '))

        # Submit the search form
        search_input.send_keys(Keys.RETURN)

        # Wait for the search results container to be visible
        time.sleep(3)  # Give time for the page to load

        # Check if search results are available
        try:
            # Find and click the first result
            first_a_tag = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'a.search-container-box.shadow.relative'))
            )
            first_a_tag.click()

            # Wait for the description and details to appear
            description_text = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.text-xs.rounded-md.my-2'))
            ).text

            # Wait for the plant's additional details
            plant_details = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.text-xs.grid.md\\:grid-cols-2.gap-2.bg-gray-100.rounded.p-3'))
            )

            # Extract plant details
            details = {}
            for detail in plant_details.find_elements(By.CSS_SELECTOR, 'div.flex.items-center.gap-1.capitalize'):
                key = detail.find_element(By.TAG_NAME, 'h3').text.strip(':')
                value = detail.find_element(By.TAG_NAME, 'p').text.strip()
                details[key] = value

            # Wait for the plant's care sections (watering, sunlight, etc.)
            care_sections = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.rounded-md.shadow.p-3'))
            )

            # Extract care info
            care_info = {}
            for section in care_sections:
                heading = section.find_element(By.CSS_SELECTOR, 'h3.font-bold.text-xl.capitalize').text.strip()
                text = section.find_element(By.CSS_SELECTOR, 'p').text.strip()
                care_info[heading] = text

            # Save the collected data
            species_data[species_name] = {
                'mapping_name': mapping,
                'description': description_text,
                'details': details,
                'care_info': care_info
            }

        except Exception as e:
            # If no results found, just save the species name with empty details
            species_data[species_name] = None
            print(f"No data found for {species_name} - Error: {e}")

    except Exception as e:
        print(f"Error occurred while processing {species_name}: {e}")
        species_data[species_name] = None

# Save the species data into a JSON file
with open('species_data.json', 'w', encoding='utf-8') as json_file:
    json.dump(species_data, json_file, ensure_ascii=False, indent=4)

# Close the driver after the loop finishes
driver.quit()

print("Species data collection complete.")

In [90]:
# Save the corrected JSON data
output_file_path = 'corrected_species_data.json'
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(species_data, file, ensure_ascii=False, indent=4)